In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install surprise
!pip install papermill
!pip install scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 13.6 MB/s 
     |████████████████████████████████| 122 kB 57.9 MB/s 
     |████████████████████████████████| 355 kB 66.3 MB/s 
     |████████████████████████████████| 5.5 MB 65.9 MB/s 
     |████████████████████████████████| 12.4 MB 41.8 MB/s 
     |████████████████████████████████| 310 kB 63.2 MB/s 
     |████████████████████████████████| 61 kB 9.2 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 396 kB 78.0 MB/s 
     |████████████████████████████████| 7.6 MB 42.9 MB/s 
     |████████████████████████████████| 182 kB 33.1 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705372 sha256=5a5172b629bff98a797e5e149daf2fc05ce6f936dedcca2d19f547a195b338b7
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a10

In [1]:
import sys
import os
import surprise
import papermill as pm
import scrapbook as sb
import pandas as pd

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Surprise version: 1.1.1


In [2]:
data_train = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_train_than30.csv')
data_test = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_test_than30.csv')
book_info = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_books_than30.csv')

In [4]:
data_train.head()

,id,product_id,user_id,user_name,rating,content,thank_count,rating_average,count
0,6081499,3734909.0,88616.0,Trương Minh Đăng,5,NaN,0,4.8,1
1,12731297,14961285.0,16848626.0,Phạm Văn Tùng,5,NaN,0,4.7,1
2,7849857,68774130.0,8046945.0,Minh Giải,5,NaN,0,5.0,1
3,15666018,152449148.0,1333575.0,Genny Lee,5,bộ truyện rất hay và giàu tính nhân văn.,0,4.9,1
4,17241576,655679.0,24504222.0,Nhật Việt,5,NaN,0,4.9,1


In [3]:
df_train = data_train[['user_id', 'product_id', 'rating']]

In [4]:
df_train.head()

,user_id,product_id,rating
0,88616.0,3734909.0,5
1,16848626.0,14961285.0,5
2,8046945.0,68774130.0,5
3,1333575.0,152449148.0,5
4,24504222.0,655679.0,5


In [5]:
df_train.columns=['userID', 'itemID','rating']

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140839 entries, 0 to 140838
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   userID  140839 non-null  float64
 1   itemID  140839 non-null  float64
 2   rating  140839 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 3.2 MB


In [7]:
df_train.shape

(140839, 3)

In [8]:
df_test = data_test[['user_id', 'product_id', 'rating']]

In [9]:
df_test.columns=['userID', 'itemID','rating']

In [10]:
df_test.shape

(1423, 3)

In [11]:
# 'reader' is being used to get rating scale (for MovieLens, the scale is [1, 5]).
# 'rating_scale' parameter can be used instead for the later version of surprise lib:
# https://github.com/NicolasHug/Surprise/blob/master/surprise/dataset.py
reader = surprise.Reader(rating_scale=(1, 5))
train_set = surprise.Dataset.load_from_df(df_train, reader=reader).build_full_trainset()
train_set

In [14]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 24.60759762499947 seconds for training.


# Prediction

In [23]:
predictions = predict(svd, df_test, usercol='userID', itemcol='itemID')
predictions.head()

,userID,itemID,prediction
0,2238735.0,10075763.0,4.990084
1,11992154.0,69764534.0,3.926843
2,22405575.0,982037.0,4.982538
3,16952419.0,75996783.0,4.507215
4,18025738.0,874800.0,5.000000


In [16]:
predictions.shape

(1423, 3)

# Remove rated movies in the top k recommendations

In [18]:
with Timer() as test_time:
    all_predictions = compute_ranking_predictions(svd, df_train, usercol='userID', itemcol='itemID', remove_seen=True)
    
print("Took {} seconds for prediction.".format(test_time.interval))

Took 477.0912014610003 seconds for prediction.


In [19]:
all_predictions.shape

(45130586, 3)

In [20]:
#Save all_predictions
all_predictions.to_csv('/content/drive/MyDrive/recomment_system/Do_an/all_predictions_svd.csv', header=True, index=None)

In [12]:
#Load all_predictions
all_predictions = pd.read_csv("/content/drive/MyDrive/recomment_system/Do_an/all_predictions_svd.csv")

#Evaluate how well SVD performs

In [21]:
eval_rmse = rmse(df_test, predictions)
eval_mae = mae(df_test, predictions)
eval_rsquared = rsquared(df_test, predictions)
eval_exp_var = exp_var(df_test, predictions)
print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

RMSE:		0.384029
MAE:		0.185945
rsquared:	0.264586
exp var:	0.270042


In [22]:
k = 10
eval_map = map_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("MAP:\t%f" % eval_map)

MAP:	0.000161


In [24]:
k=10
eval_ndcg = ndcg_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("NDCG:\t%f" % eval_ndcg)

NDCG:	0.000344


In [13]:
k=10
eval_precision = precision_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("Precision@K:\t%f" % eval_precision)


Precision@K:	0.000097


In [25]:
k=10
eval_recall = recall_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("Recall@K:\t%f" % eval_recall)

Recall@K:	0.000967
